### Plot candles with a list of moving averages

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [3]:
df = pd.read_pickle(utils.get_data_filename(pair, granularity, True))

In [4]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [6]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [7]:
df_ma.dropna(inplace=True)

In [8]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2024-01-04T19:00:00.000000000Z,1.09474,1.09500,1.09434,1.09470,1.095012,1.094680,1.093200,1.094859,1.101402,1.100299
256,2024-01-04T20:00:00.000000000Z,1.09470,1.09494,1.09431,1.09494,1.094975,1.094825,1.093302,1.094747,1.101309,1.100309
257,2024-01-04T21:00:00.000000000Z,1.09492,1.09502,1.09448,1.09449,1.094947,1.094939,1.093375,1.094608,1.101211,1.100316
258,2024-01-04T22:00:00.000000000Z,1.09487,1.09510,1.09456,1.09478,1.094975,1.095034,1.093475,1.094482,1.101089,1.100324
259,2024-01-04T23:00:00.000000000Z,1.09472,1.09490,1.09434,1.09463,1.094863,1.095088,1.093605,1.094374,1.100963,1.100331


In [9]:
df_plot = df_ma.iloc[-300:].copy()

In [10]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
    ))
fig.update_layout(width=1200,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()